In [1]:
from pathlib import Path
from tqdm.auto import tqdm
from tabulate import tabulate
import random, json
import pycountry
from iso639 import languages
import networkx as nx
import textwrap
from itertools import combinations, islice
from dateutil import parser
from datetime import datetime
from collections import Counter
from langdetect import detect
from omnibelt import load_json, save_json
import Levenshtein
from unidecode import unidecode
import unicodedata
from wordcloud import WordCloud
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from rake_nltk import Rake
import spacy
import pickle as pkl

# from sklearn.cluster import DBSCAN
# import numpy as np

stop_words = set(stopwords.words('english'))
stop_words.update("'s", '|', 'I', "n't", "`s", "'s", 'n`t')

nlp = spacy.load('en_core_web_md')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import umap
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import nltk

# Download the tokenizers
# nltk.download('punkt')

# word_vectors = KeyedVectors.load_word2vec_format('/home/fleeb/Downloads/GoogleNews-vectors-negative300.bin',
# 												 binary=True)


In [2]:
lang_clusters = {'en': ['au', 'ca', 'gb', 'ie', 'in', 'my', 'ng', 'nz', 'ph', 'sa', 'sg', 'us', 'za'],
				 'es': ['ar', 'co', 'cu', 'mx', 've'], 'de': ['at', 'ch', 'de'], 'fr': ['be', 'fr', 'ma'],
				 'zh': ['cn', 'hk', 'tw'], 'ar': ['ae', 'eg'], 'pt': ['br', 'pt'], 'bg': ['bg'], 'cs': ['cz'],
				 'el': ['gr'], 'he': ['il'], 'hu': ['hu'], 'id': ['id'], 'it': ['it'], 'ja': ['jp'], 'ko': ['kr'],
				 'lt': ['lt'], 'lv': ['lv'], 'nl': ['nl'], 'no': ['no'], 'pl': ['pl'], 'ro': ['ro'], 'ru': ['ru'],
				 'sv': ['se'], 'sl': ['si'], 'sk': ['sk'], 'sr': ['rs'], 'th': ['th'], 'tr': ['tr'], 'uk': ['ua']}
loc_names = {'gb': 'United Kingdom', 'ar': 'Argentina', 'pl': 'Poland', 'sk': 'Slovakia', 'us': 'United States',
			 'eg': 'Egypt', 'no': 'Norway', 'ph': 'Philippines', 'at': 'Austria', 'rs': 'Serbia', 'tw': 'Taiwan',
			 'be': 'Belgium', 'cu': 'Cuba', 'sa': 'Saudi Arabia', 'th': 'Thailand', 'id': 'Indonesia',
			 'ru': 'Russian Federation', 'ch': 'Switzerland', 'fr': 'France', 'lt': 'Lithuania', 'tr': 'Turkey',
			 'de': 'Germany', 'cz': 'Czechia', 'pt': 'Portugal', 'ae': 'United Arab Emirates', 'it': 'Italy',
			 'cn': 'China', 'lv': 'Latvia', 'nl': 'Netherlands', 'hk': 'Hong Kong', 'ca': 'Canada', 'br': 'Brazil',
			 'hu': 'Hungary', 'kr': 'Korea', 'si': 'Slovenia', 'au': 'Australia', 'my': 'Malaysia', 'ie': 'Ireland',
			 'ua': 'Ukraine', 'in': 'India', 'ma': 'Morocco', 'bg': 'Bulgaria', 'ng': 'Nigeria', 'il': 'Israel',
			 'se': 'Sweden', 'za': 'South Africa', 've': 'Venezuela', 'nz': 'New Zealand', 'jp': 'Japan',
			 'sg': 'Singapore', 'gr': 'Greece', 'mx': 'Mexico', 'co': 'Colombia', 'ro': 'Romania'}
lang_names = {'en': 'English', 'ko': 'Korean', 'ru': 'Russian', 'es': 'Spanish', 'pt': 'Portuguese', 'cs': 'Czech',
			  'tr': 'Turkish', 'nl': 'Dutch', 'ar': 'Arabic', 'fr': 'French', 'bg': 'Bulgarian', 'id': 'Indonesian',
			  'sk': 'Slovak', 'el': 'Greek', 'he': 'Hebrew', 'sr': 'Serbian', 'hu': 'Hungarian', 'th': 'Thai',
			  'zh': 'Chinese', 'no': 'Norwegian', 'sl': 'Slovenian', 'sv': 'Swedish', 'de': 'German', 'lv': 'Latvian',
			  'pl': 'Polish', 'it': 'Italian', 'ro': 'Romanian', 'lt': 'Lithuanian', 'ja': 'Japanese',
			  'uk': 'Ukrainian'}
cluster_id = {loc: lang for lang, locs in lang_clusters.items() for loc in locs}
len(cluster_id)

54

In [3]:
# root = Path('/home/fleeb/workspace/local_data/nnn')
# save_json(lang_clusters, root / 'language_clusters.json')
# save_json(loc_names, root / 'location_names.json')
# save_json(lang_names, root / 'language_names.json')

In [5]:
root = Path('/home/fleeb/workspace/local_data/nnn')
recs = (root / 'babel-briefings-v1-anon').glob('**/*.json')
recs = list(recs)
len(recs)
articles = []
for rec in tqdm(recs):
	articles.extend(load_json(rec))
by_loc = {}
by_lang = {}
by_source = {}
by_ID = {}
for article in tqdm(articles):
	by_ID[article['ID']] = article
	# article['published'] = parser.parse(article['publishedAt'])
	by_source.setdefault(article['source-name'], []).append(article)
	by_lang.setdefault(article['language'], []).append(article)
	for instance in article['instances']:
		# instance['collected'] = parser.parse(instance['collectedAt'])
		by_loc.setdefault(instance['location'], []).append(article)
len(articles), len(by_loc), len(by_lang)

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/4719199 [00:00<?, ?it/s]

(4719199, 54, 30)

In [29]:
random.choice(articles)

{'author': 'author#129220',
 'title': 'Έφτασαν τα Soundmojis στο Messenger: Τι είναι και πώς ενεργοποιούνται - Τεχνολογία | News 24/7 - News247.gr',
 'description': 'Οι χρήστες του Messenger πλέον έχουν τη δυνατότητα να στέλνουν ηχητικά emojis. Πώς το ενεργοποιείτε.',
 'url': 'https://www.news247.gr/technologia/eftasan-soundmojis-sto-messenger-ti-einai-kai-pos-energopoioyntai.9299041.html',
 'urlToImage': 'https://www.news247.gr/img/9468/9299045/824000/fb1200/1200/sound.jpg',
 'publishedAt': '2021-07-19T05:05:17Z',
 'content': 'Facebook Messenger, Soundmojis emojis .\r\n Soundmoji , Fast &amp; Furious 9. Techgear.gr, Soudmojis . , Emoji 17 .\r\n Soundmojis, Messenger Facebook . smiley , , . Soundmojis. \r\n Statista.com, Messenge… [+56 chars]',
 'instances': [{'collectedAt': '2021-07-19T13:19:00Z',
   'location': 'gr',
   'category': 'general'},
  {'collectedAt': '2021-07-19T13:19:00Z',
   'location': 'gr',
   'category': 'technology'},
  {'collectedAt': '2021-07-20T08:50:00Z',
   'loc

In [5]:
IDs = list(by_ID.keys())

NameError: name 'by_ID' is not defined

In [6]:
random.seed(0)
# art_IDs = random.sample(IDs, 100)
# arts = random.sample(articles, 100)
arts = [art for lang in lang_clusters for art in random.sample(by_lang[lang], 4, counts=[len(a['instances']) if len(a['instances']) <= 10 else 0 for a in by_lang[lang]])]
assert len(arts) == len(set(art['ID'] for art in arts))
# arts = [by_ID[ID] for ID in art_IDs]
len(json.dumps(arts))

NameError: name 'by_lang' is not defined

In [7]:
arts = load_json(root / 'sample_articles.json')

In [8]:
print(tabulate(Counter([lang_names[art['language']] for art in arts]).most_common(), headers=['Language', 'Count']))

Language      Count
----------  -------
English           4
Spanish           4
German            4
French            4
Chinese           4
Arabic            4
Portuguese        4
Bulgarian         4
Czech             4
Greek             4
Hebrew            4
Hungarian         4
Indonesian        4
Italian           4
Japanese          4
Korean            4
Lithuanian        4
Latvian           4
Dutch             4
Norwegian         4
Polish            4
Romanian          4
Russian           4
Swedish           4
Slovenian         4
Slovak            4
Serbian           4
Thai              4
Turkish           4
Ukrainian         4


In [9]:
print(tabulate(Counter([len(art['instances']) for art in arts]).most_common(), headers=['Instances', 'Count']))

  Instances    Count
-----------  -------
          1       60
          2       42
          3       14
          9        2
          6        2


In [10]:
print(tabulate(Counter([loc_names[inst['location']] for art in arts for inst in art['instances']]).most_common(), headers=['Locations', 'Count']))

Locations               Count
--------------------  -------
Austria                     9
Bulgaria                    9
Indonesia                   9
Turkey                      9
Czechia                     8
Israel                      8
Hungary                     8
Korea                       8
Sweden                      8
Japan                       7
Norway                      7
Romania                     7
Slovenia                    7
Ukraine                     7
Greece                      6
Lithuania                   6
Slovakia                    6
Thailand                    6
Belgium                     5
Hong Kong                   5
United Arab Emirates        5
Egypt                       5
Latvia                      5
Poland                      5
Russian Federation          5
Morocco                     4
France                      4
Italy                       4
Netherlands                 4
Serbia                      4
Argentina                   3
Switzerlan

In [12]:
print(tabulate(Counter([inst['category'] for art in arts for inst in art['instances']]).most_common(), headers=['Category', 'Count']))

Category         Count
-------------  -------
technology          55
science             43
sports              41
entertainment       31
business            27
general             14
health               5


In [84]:
# save_json(arts, root / 'sample_articles.json')

In [7]:
authors = {}
with tqdm(articles) as pbar:
	for article in pbar:
		authors.setdefault(article.get('author'), []).append(article['ID'])
		# pbar.set_description(f'{len(authors)} authors')
len(authors)

  0%|          | 0/4719199 [00:00<?, ?it/s]

233018

In [8]:
existing = [author for author in authors if not isinstance(author, str) or not author.startswith('author#')]
len(existing)

3

In [9]:
existing

[None, '', '""']

In [8]:
author_keys = list(authors.keys())
author_keys.remove('')
author_keys.remove(None)
author_keys.remove('""')
random.seed(0)
random.shuffle(author_keys)
len(author_keys)

233015

In [9]:
anon_authors = {key: f'author#{str(anonID).zfill(6)}' for key, anonID in zip(author_keys, range(len(author_keys)))}
deanon_authors = {anonID: key for key, anonID in anon_authors.items()}
assert len(anon_authors) == len(deanon_authors)

In [10]:
anonroot = Path('/home/fleeb/workspace/local_data/nnn') / 'babel-briefings-v1-anon'
anonroot.mkdir(exist_ok=True)

In [13]:
save_json(deanon_authors, anonroot / 'authors_anonymization.json')
for rec in tqdm(recs):
	raw = load_json(rec)
	# for article in tqdm(raw, desc=f'{rec.name} ({i+1}/{len(recs)})'):
	for article in raw:
		if 'author' in article and article['author'] in anon_authors:
			article['author'] = anon_authors[article['author']]
	save_json(raw, anonroot / rec.name)

  0%|          | 0/54 [00:00<?, ?it/s]

In [19]:
import nltk
from nltk import ne_chunk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag


In [20]:

nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/fleeb/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/fleeb/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/fleeb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fleeb/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [40]:

text = "Index"
text = random.choice(list(authors))
tokens = word_tokenize(text)
tags = pos_tag(tokens)
tree = ne_chunk(tags)

print(text)
print(tree)


Wayne Chang and Jill Disis, CNN Business
(S
  (PERSON Wayne/NNP)
  (PERSON Chang/NNP)
  and/CC
  (PERSON Jill/NNP Disis/NNP)
  ,/,
  (ORGANIZATION CNN/NNP Business/NNP))
